In [ ]:
import os
import shutil
import pandas as pd
import zipfile
from datetime import datetime

# Função para padronizar datas no formato americano (yyyy-mm-dd), ignorando a hora, se houver
def padronizar_data(data):
    try:
        # Remove o horário (T) e o sufixo Z se existirem
        if 'T' in data:
            data = data.split('T')[0]
        if 'Z' in data:
            data = data.replace('Z', '')
        # Tenta converter para o formato %Y-%m-%d
        return datetime.strptime(data, '%Y-%m-%d').strftime('%Y-%m-%d')
    except ValueError:
        try:
            return datetime.strptime(data, '%d/%m/%Y').strftime('%Y-%m-%d')
        except ValueError:
            return data  # Se não puder converter, retorna o valor original

# Função para concatenar colunas de year, month, day em uma nova coluna "date"
def concatenar_data(row):
    year = str(row.get('year', ''))
    month = str(row.get('month', '')).zfill(2)  # Preenche com zero à esquerda se necessário
    day = str(row.get('day', '')).zfill(2)

    # Se ano existir, cria data; caso contrário, retorna vazio
    if year:
        return f"{year}-{month or '01'}-{day or '01'}"  # Usa '01' se mês ou dia estiverem ausentes
    return ''

# Função para verificar e converter coordenadas
def converter_coordenadas(valor):
    try:
        return float(str(valor).replace(',', '.'))
    except ValueError:
        return None

# Função para processar os arquivos CSV
def processar_csv(file_path, output_path):
    df = ler_csv_arquivo(file_path)
    if df is not None:
        # Procura colunas que contenham "date" ou "data" e que não sejam "datasetKey"
        data_columns = [col for col in df.columns if ('date' in col.lower() or 'data' in col.lower()) and col.lower() != 'datasetkey']

        if data_columns:
            print(f"Colunas de data encontradas em {file_path}: {data_columns}")
            for col in data_columns:
                # Cria uma nova coluna com o nome "nomedacoluna_pad"
                nova_coluna = f"{col}_pad"
                df[nova_coluna] = df[col].apply(lambda x: padronizar_data(str(x)))

        # Se não encontrar colunas "date" ou "data", verifica se existem "year", "month", "day"
        elif any(col in df.columns for col in ['year', 'month', 'day']):
            print(f"Colunas 'year', 'month' ou 'day' encontradas em {file_path}, criando coluna 'date_pad'")
            df['date_pad'] = df.apply(concatenar_data, axis=1)

        # Verifica colunas de latitude e longitude
        lat_columns = [col for col in df.columns if 'lat' in col.lower() or 'latitude' in col.lower()]
        long_columns = [col for col in df.columns if 'long' in col.lower() or 'longitude' in col.lower()]

        if lat_columns and long_columns:
            print(f"Colunas de coordenadas encontradas em {file_path}: {lat_columns}, {long_columns}")
            # Usa a primeira coluna encontrada de latitude e longitude
            df['Lat_pad'] = df[lat_columns[0]].apply(converter_coordenadas)
            df['Long_pad'] = df[long_columns[0]].apply(converter_coordenadas)

        # Salva o novo arquivo .csv com o prefixo "pad_"
        new_file_name = os.path.basename(output_path)
        df.to_csv(os.path.join(output_folder, new_file_name), encoding='utf-8', index=False)
        print(f"Arquivo processado e salvo: {new_file_name}")
    else:
        print(f"Erro ao processar {file_path}")

# Função para ler CSV com autodetecção de delimitador
def ler_csv_arquivo(file_path):
    try:
        # Tenta diferentes delimitadores (vírgula, ponto e vírgula, tabulação)
        df = pd.read_csv(file_path, sep=None, engine='python', encoding='utf-8')
        print(f"Lendo arquivo CSV: {file_path}")
        return df
    except Exception as e:
        print(f"Erro ao ler {file_path}: {e}")
        return None

# Caminhos das pastas
input_folder = input("Insira a pasta de input: ")
output_folder = input("Insira a pasta de output: ")
temp_folder = os.path.join(output_folder, 'temp_extracted')

# Cria a pasta de saída e temporária, se não existirem
os.makedirs(output_folder, exist_ok=True)
os.makedirs(temp_folder, exist_ok=True)

# Percorre os arquivos da pasta de entrada
for root, dirs, files in os.walk(input_folder):
    for file in files:
        print(f"Processando arquivo: {file}")
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            output_path = os.path.join(output_folder, file)
            processar_csv(file_path, output_path)

        elif file.endswith('.zip'):
            zip_path = os.path.join(root, file)
            print(f"Processando arquivo ZIP: {file}")

            # Extrai arquivos .csv do zip para a pasta temporária
            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(temp_folder)
                    print(f"Arquivos extraídos de {file}")
            except Exception as e:
                print(f"Erro ao extrair arquivos do zip {file}: {e}")
                continue

            # Processa os arquivos .csv extraídos na pasta temporária
            for extracted_file in os.listdir(temp_folder):
                extracted_file_path = os.path.join(temp_folder, extracted_file)
                if os.path.isfile(extracted_file_path) and extracted_file.endswith('.csv'):
                    output_path = os.path.join(output_folder, extracted_file)
                    processar_csv(extracted_file_path, output_path)

# Limpa a pasta temporária após o processamento
for temp_file in os.listdir(temp_folder):
    temp_file_path = os.path.join(temp_folder, temp_file)
    if os.path.isfile(temp_file_path):
        os.remove(temp_file_path)
    elif os.path.isdir(temp_file_path):
        shutil.rmtree(temp_file_path)

print("Processamento concluído!")
